In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# URL of the page
url_df = 'https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures'

# Request page content and parse with BeautifulSoup
response = requests.get(url_df)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup)
# Locate the table and read it into pandas
table = soup.find('table')
df = pd.read_html(str(table))[0]

# Flatten multi-level columns and clean them up
df.columns = [' '.join(col).strip() for col in df.columns]
df = df.reset_index(drop=True)

# Remove spaces in column names and drop fully empty rows
df.columns = [col.replace(" ", "") for col in df.columns]
df = df.dropna(how='all')

# Extract hyperlinks in "Match Report" column, if available
match_report_links = []
for row in table.find_all('tr')[1:]:  # Skip header row
    link = row.find('td', {'data-stat': 'match_report'})
    if link and link.find('a'):
        match_report_links.append('https://fbref.com' + link.find('a')['href'])
# Add hyperlinks as a new column
df['MatchReportURL'] = match_report_links
df['id'] = range(0, len(df))
# Display the first few rows to check data
df.head()



<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202411012" rel="dns-prefetch"/>
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp.osano.com/16CGnCU8UtNhM14sg/126698

C:\Users\theja\AppData\Local\Temp\ipykernel_33316\482960713.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,MatchReport,Notes,MatchReportURL,id
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN,https://fbref.com/en/matches/e62f6e78/Crystal-...,0
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN,https://fbref.com/en/matches/6713c1dc/Fulham-L...,1
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN,https://fbref.com/en/matches/09d8a999/Tottenha...,2
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,Match Report,NaN,https://fbref.com/en/matches/1ac96eb4/Newcastl...,3
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN,https://fbref.com/en/matches/82702941/Leeds-Un...,4


In [2]:
df = df.drop('MatchReport', axis=1)
df = df.drop('Notes', axis=1)

df['Split Score'] = df['Score'].str.split('–')
df['Home Score'] = df['Split Score'].str[0]
df['Away Score'] = df['Split Score'].str[1]
df = df.drop('Split Score', axis=1)

def get_winner(row):
    # Replace any non-standard dash with a standard hyphen and split scores
    home_score = row['Home Score']
    away_score = row['Away Score']    
    
    # Determine the winner
    if home_score > away_score:
        return 'Home'
    elif home_score < away_score:
        return 'Away'
    else:
        return "Draw"
    
# Apply the function to each row and create a new column 'Winner'
df["Winner"] = df.apply(get_winner, axis=1)

In [3]:
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,MatchReportURL,id,Home Score,Away Score,Winner
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,https://fbref.com/en/matches/e62f6e78/Crystal-...,0,0,2,Away
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,https://fbref.com/en/matches/6713c1dc/Fulham-L...,1,2,2,Draw
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,https://fbref.com/en/matches/09d8a999/Tottenha...,2,4,1,Home
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,https://fbref.com/en/matches/1ac96eb4/Newcastl...,3,2,0,Home
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,https://fbref.com/en/matches/82702941/Leeds-Un...,4,2,1,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,39201.0,Goodison Park,Stuart Attwell,https://fbref.com/en/matches/94de848f/Everton-...,375,1,0,Home
426,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,32183.0,King Power Stadium,Simon Hooper,https://fbref.com/en/matches/a96c9915/Leiceste...,376,2,1,Home
427,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,NaN,Villa Park,David Coote,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,377,2,1,Home
428,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,36871.0,Elland Road,Anthony Taylor,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,378,1,4,Away


In [4]:
from pathlib import Path

filepath = Path('2023.csv')  
df.to_csv(filepath, index=False)

<Response [429]>


In [361]:
merged_df = pd.merge(df, sub_df, on='id', how='inner')

In [364]:
merged_df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,...,ACrs,AInt,ATklW,APKwon,APKcon,AOG,ARecov,AWon,ALost,AWon%
0,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,...,11,4,4,0,0,0,43,14,9,60.9


In [249]:
table = soup.find('table', id='keeper_stats_943e8050')

# Extract and print headers (from the first <thead>)
headers = [header.text.strip() for header in table.find_all('th')]
print("Headers:", headers)

# Extract and print subheaders (from the second <thead>)
subheaders = []
second_header = table.find_all('thead')[0]  # Access the second <thead>
for th in second_header.find_all('th'):
    subheaders.append(th.text.strip())

print("Subheaders:", subheaders)


Headers: ['', 'Shot Stopping', 'Launched', 'Passes', 'Goal Kicks', 'Crosses', 'Sweeper', 'Player', 'Nation', 'Age', 'Min', 'SoTA', 'GA', 'Saves', 'Save%', 'PSxG', 'Cmp', 'Att', 'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen', 'Att', 'Launch%', 'AvgLen', 'Opp', 'Stp', 'Stp%', '#OPA', 'AvgDist', 'James Trafford']
Subheaders: ['', 'Shot Stopping', 'Launched', 'Passes', 'Goal Kicks', 'Crosses', 'Sweeper', 'Player', 'Nation', 'Age', 'Min', 'SoTA', 'GA', 'Saves', 'Save%', 'PSxG', 'Cmp', 'Att', 'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen', 'Att', 'Launch%', 'AvgLen', 'Opp', 'Stp', 'Stp%', '#OPA', 'AvgDist']
